### Data Engineering

In [1]:
# Add all the imports
import pandas as pd
import numpy as np

In [2]:
def map_idx(unique_attribute_vals: list):

    # Initialize the variables
    value_to_idx = {}
    idx_to_value = {}
    unique_int = 1

    # Loop through to map the values
    for value in unique_attribute_vals:
        value_to_idx[value] = unique_int
        idx_to_value[unique_int] = value
        unique_int = unique_int + 1

    # Return the two dictionaries
    return (value_to_idx, idx_to_value)

In [3]:
data_path = "input_data/NY-House-Dataset.csv"
all_data = pd.read_csv(data_path).dropna()

In [4]:
all_data.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [5]:
# Get unique values for the columns listed below.
attribute_list = ["BROKERTITLE", "TYPE", "STATE", "ADMINISTRATIVE_AREA_LEVEL_2", "LOCALITY", "SUBLOCALITY", "STREET_NAME", "LONG_NAME"]
attribute_idx_mapping = {}

for attribute in attribute_list:
    unique_attribute_values = all_data[attribute].unique()
    value_to_idx, idx_to_value = map_idx(unique_attribute_values)
    attribute_idx_mapping[attribute] = {"value_to_idx": value_to_idx, "idx_to_value": idx_to_value}
    all_data[attribute] = all_data[attribute].map(value_to_idx)

In [6]:
all_data.drop(columns=["ADDRESS", "MAIN_ADDRESS", "FORMATTED_ADDRESS", "LATITUDE", "LONGITUDE"], inplace=True)
all_data.to_csv("input_data/model_ready_data.csv")
all_data.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,STATE,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME
0,1,1,315000,2,2.000000,1400.0,1,1,1,1,1,1
1,2,1,195000000,7,10.000000,17545.0,2,2,1,2,2,2
2,3,2,260000,4,2.000000,2015.0,3,2,1,3,3,3
3,4,1,69000,3,1.000000,445.0,4,2,1,2,2,4
4,5,3,55000000,7,2.373861,14175.0,5,2,1,2,2,5
